In [3]:
import pandas as pd
import numpy as np

data_df = pd.read_csv(r'D:\COMP 551 P1\Adult Dataset\adult.data')

In [35]:
data_np = data_df.to_numpy()

check = data_np[242][1]
print(data_np.shape)
print(type('?'))
print(type(data_np[242][1]))
print(data_np[242][1])
print(data_np[242][1] == check)

(32560, 15)
<class 'str'>
<class 'str'>
 ?
True


In [36]:
for x in data_np:
    for i in range(15):
        if x[i] == ' ?':
            print(x[i])
            np.delete(data_np, np.where(data_np = x))
            
print(data_np.shape)

 ?


TypeError: where() got an unexpected keyword argument 'data_np'